<a href="https://colab.research.google.com/github/RedPanda54/Machine_Learning_study/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
import seaborn as sns
URL = "https://raw.githubusercontent.com/RedPanda54/Machine_Learning_study/main/diabetes_prediction_dataset.csv"
df = pd.read_csv(URL)
df.info() # 데이터에 결측치는 없는 것으로 파악된다.

In [ ]:
df.head()

In [ ]:
# 데이터 shape 파악
df.shape

In [ ]:
# 데이터 통계량 파악
df.describe()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.diabetes.value_counts(normalize = True).mul(100)